In [4]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import string
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import torch
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve
import matplotlib.pyplot as plt
import time
from transformers import Trainer, TrainingArguments, AlbertForSequenceClassification, AlbertTokenizer
from datasets import load_dataset

from transformers import Trainer, TrainingArguments, RobertaTokenizer
from datasets import Dataset
import pandas as pd
import time
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve
from transformers import Trainer, TrainingArguments, RobertaTokenizer
from datasets import Dataset
import pandas as pd
import time
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve
import torch
from transformers import Trainer, TrainingArguments, RobertaForSequenceClassification, RobertaTokenizer
from torch.quantization import quantize_dynamic
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import pandas as pd
import numpy as np
import os
import copy
from transformers import Trainer, TrainingArguments, RobertaForSequenceClassification, RobertaTokenizer
from datasets import load_dataset
import torch.nn.utils.prune as prune
import time

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
models_list = ["roberta/fine_tune/prune/pruned_model.pth","albert/checkpoint-9900",
               "roberta/checkpoint-9900","roberta/fine_tune/distilled_model/checkpoint-3300","roberta-base_20_epochs/checkpoint-6600"]#"roberta/fine_tune/quant/quantized_model.pth","albert/fine_tune/quantize/quantized_model.pth"]

In [6]:
models_list = ["roberta-base_20_epochs/checkpoint-6600"]

In [7]:

# Load data
export_dir = Path(os.getcwd())
data_path = Path(export_dir, "data")
train_file = Path(data_path, 'train_data_only_text_and_labels.csv')
eval_file = Path(data_path, 'eval_data_only_text_and_labels.csv')
test_file = Path(data_path, 'test_data_only_text_and_labels.csv')

train_df = pd.read_csv(train_file)
eval_df = pd.read_csv(eval_file)
test_df = pd.read_csv(test_file)

## Define eval functions:

In [8]:
def metric_fn(eval_pred):

    
    # Extract predictions and labels
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Convert logits to predicted class
    
    # Compute basic metrics
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    
    # Compute AUC
    if len(np.unique(labels)) > 1:  # Check if there is more than one class in labels
        auc = roc_auc_score(labels, logits[:, 1])  # Use probability of the positive class
    else:
        auc = float('nan')  # If only one class is present, AUC is not defined

    # Compute running time

    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc,
    }

In [9]:
for model_str in models_list:
    ### SETUP PARAMS
    checkpoint_path = f"./results/{model_str}"


    # LOAD TOKENIZER
    print(model_str)
    if "roberta" in model_str:
        if "checkpoint" in model_str:
            model = RobertaForSequenceClassification.from_pretrained(checkpoint_path)
        else:
            model = torch.load(checkpoint_path)
        if "distilled" in model_str:
            tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
        else:
            tokenizer = RobertaTokenizer.from_pretrained("roberta-large")
    if "albert" in model_str:
        if "checkpoint" in model_str:
            model = AlbertForSequenceClassification.from_pretrained(checkpoint_path)
        else:
            model = AlbertForSequenceClassification.from_pretrained("albert-base-v2")
            model.load_state_dict(torch.load(checkpoint_path))
        if "distilled" in model_str:
            tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")
        else:
            tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")

    


    test_dataset = Dataset.from_pandas(test_df)
    

    
    # Define a function to tokenize the inputs
    def tokenize_function(examples):
        return tokenizer(examples['text_combined'], padding='max_length', truncation=True)
    
    # Apply the tokenizer to the dataset
    tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)
    
    # Define evaluation arguments
    eval_args = TrainingArguments(
        output_dir='./results/roberta/fine_tune/prune',
        per_device_eval_batch_size=50,
        logging_dir='./results/roberta/fine_tune/prune',
        evaluation_strategy="epoch"  # Ensure evaluation is done at the end of each epoch
    )
    
    # Define the Trainer with the quantized model
    trainer = Trainer(
        model=model,
        args=eval_args,
        eval_dataset=tokenized_test_dataset,
        compute_metrics=metric_fn  
    )
    
    # Evaluate the model on the tokenized test dataset

    evaluation_results = trainer.evaluate()

    # Print evaluation results
    print(evaluation_results)
    
    with open(f'./results/{model_str}.txt', 'w') as f:
        f.write(f"{evaluation_results}\n\n")
 

roberta-base_20_epochs/checkpoint-6600


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 16498/16498 [00:36<00:00, 449.33 examples/s]
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


{'eval_loss': 0.031802643090486526, 'eval_model_preparation_time': 0.0065, 'eval_accuracy': 0.9923627106315918, 'eval_precision': 0.993807687813997, 'eval_recall': 0.9914908497493881, 'eval_f1': 0.9926479169097912, 'eval_auc': 0.999568512645248, 'eval_runtime': 108.8557, 'eval_samples_per_second': 151.558, 'eval_steps_per_second': 3.032}
